In [1]:
#import required libraries
import pyautogui as pg
import cv2 as cv
import imutils
from imutils.video import VideoStream
import numpy as np

# Start video stream (basically capturing video frame be frame)
cam = VideoStream(scr = 0).start()

# Initialize key
key = False

# Define the color range for the object to track (upper and lower RGB range for the color to detect , in out case it's blue)
colourLower = np.array([18,121,139])
colourUpper = np.array([180,255,255])


# Declaring the dimensions of the image (screen dimensions visible to us)
width = 640
height = 480

h1 = height*5//16 
h2 = height*11//16

w1 = width*5//16 
w2 = width*11//16 

# Main loop
while True:
     
    # Read a frame from the video stream
    img = cam.read()
    img = np.flip(img, axis=1)
    img = np.array(img)

    # Convert the image to HSV color space 
    hsv = cv.cvtColor(img,cv.COLOR_BGR2HSV)
    
    # Apply Gaussian blur to the image (reducing the quality of image is make it easier to process)
    blurred = cv.GaussianBlur(hsv,(11,11),0)
    
    # Create a mask based on the color range
    mask = cv.inRange(blurred,colourLower,colourUpper)
    
    # Apply morphological opening to the mask (mathematical operations to reduce extra noise)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN,np.ones((5,5),np.uint8))
    
    # Apply morphological closing to the mask
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE,np.ones((5,5),np.uint8))
    
    # Define the regions of interest(ROI) for the different keys (defining up, down, left and right regions)
    A_Contour = mask[h1:h2, 0:w1-20]
    D_Contour = mask[h1:h2, w2+20:width]
    W_Contour = mask[0:h1-20, w1:w2]
    S_Contour = mask[h2+20:height, w1:w2]
    
    
    # Find the contours for each ROI ( checking if the object is in the region or not)(here each function returns a boolean value)
    cnts_W = cv.findContours(W_Contour,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE) 
    cnts_A = cv.findContours(A_Contour,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    cnts_S = cv.findContours(S_Contour,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    cnts_D = cv.findContours(D_Contour,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)

    
    # Draw rectangles and text on the image for each ROI
    img = cv.rectangle(img, (0,h1),(w1-20,h2),(0,255,0),2)
    cv.putText(img,'LEFT',(40,230),cv.FONT_HERSHEY_DUPLEX, 1, (139,0,0))

    img = cv.rectangle(img, (w2+20,h1),(width,h2),(0,255,0),2)
    cv.putText(img,'RIGHT',(510,230),cv.FONT_HERSHEY_DUPLEX,  1, (139,0,0))
    
    img = cv.rectangle(img, (w1,0),(w2,h1-20),(0,255,0),2)
    cv.putText(img,'UP',(300,70),cv.FONT_HERSHEY_DUPLEX,  1, (139,0,0))
    
    img = cv.rectangle(img, (w1,h2+20),(w2,height),(0,255,0),2)
    cv.putText(img,'DOWN',(270,420),cv.FONT_HERSHEY_DUPLEX,  1, (139,0,0))
    
    #Checking if a contour is present in any ROI or not
    #If yes then corresponding key is pressed
    if len(cnts_W[0]) > 0:
        if not key:
            pg.press('W')
            key = True
        
        
    elif len(cnts_A[0]) > 0:
        if not key:
            pg.press('A')
            key = True
    
    
    elif len(cnts_S[0]) > 0:
        if not key:
            pg.press('S')
            key = True
    
    
    elif len(cnts_D[0]) > 0:
        if not key:
            pg.press('D')
            key = True
    
    else:
        key = False
    
    cv.imshow("Control", img)

    #Breaking the loop if key 'q' is pressed
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
        
#Destroying all open windows                
cam.stop()
cv.destroyAllWindows()